# Deep Direct Reinforcement Learning for Financial Signal Representation and Trading

## Abstract
資産取引において練達したトレーダーを打ち負かすことができるほどにコンピューターを訓練することはできるのか。この論文では再帰型深層ニューラルネットワークを

このモデルにおける取引＝市場の状況把握と最適行動の二つ逐次的意思決定からなる。  
従来手法と比べてこの動的な意思決定モデルは熟練したトレーダーの情報がない分だけ挑戦的。  
・・・状況を探索して各期の最適行動を決定する必要がある。  

自己学習と強化学習(RL)。
確率的最適制御と強化学習の研究。いくつかの事例では強化学習を備えたコンピューターが人間の能力を凌駕することもある。 
・・・トレーディングにおいてもRLが人間に勝つことができるのではないか？  
but トレーディングにおけるRLには従来のRLが対象にしてきた課題にはない問題がある。  

①金融環境の把握と表現の困難さから生じる問題。  
金融データには膨大な数のノイズ、ジャンプ、そして時系列を非定常なものに導くような運動が含まれている。  
移動平均や

②トレーディングのダイナミックな意思決定から生じる問題。

In [ ]:
この二つの問題に取り組むためにRDNN構造を、①の環境への反応問題と、②の意思決定問題に対処するために導入する。